### Making a Simplified K-means Clustering Algorithm

K-means clustering is a simple unsupervised machine-learning method for cluster analysis. The aim of the method is to partition a set of points into _k_ clusters, such that each point in the data is assigned to the nearest cluster. The algorithm iterates through two steps:

1. Assign each data point to the cluster with the nearest centroid
2. Update the centroids of the clusters given the new partition of the data across the clusters

The algorithm converges (finishes running) when the assignments to clusters no longer change. Since the intial assignment to clusters is (mostly) random, there is no guarantee that the optimum assignment is found. So it is common to run the algorithm multiple times and use different starting conditions.

Here, we will implement a **much** simplified version of the k-means clustering algorithm. Rather than running the algorithm until convergence, we will repeat the above two steps a large but fixed number of times. In addition, we will initialize only once, using a naive method according to which we randomly choose k points from the data to use as initial cluster centroids. 

In real life, there are plenty of libraries that allow you to implement this algorithm, and these account for much more complexity than we are going to here (such as the [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)), but this should give an idea of the intuition behind this process. Understanding this can also be very helpful for understanding how cross-validation works in machine learning (related but different idea).

For this example, we can use the `Wholesale customers data.csv`, which you can find in the repository. The file contains information on the annual spending on diverse product categories for the clients of a wholesale distributor. The data are obtained from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php) and you can find more information about them [here](http://archive.ics.uci.edu/ml/datasets/Wholesale+customers#).


### Step 1: Function to estimate Euclidean distance between two points

To start off our algorithm, we're going to make a function that calculates the [Euclidean](https://en.wikipedia.org/wiki/Euclidean_distance) distance between two n-dimensional points. The function will take two lists as arguments, where each list is the n-coordinates of each of the two points. We can call this function `find_distance`. 

We'll test the function for the points [0, 3, 0] and [4, 0, 0].

In [ ]:
# Enter your answer to Problem 1 below. 
# Formula for distance between points: sqrt((p1-q1)^2 +(p2-q2)^2)
# initialize a sum with value of 0 in order to add onto it every time coordinates have been added
# using formula to square the difference between point coordinates on dimension i 
# storing each result in "summed" and taking the square root
import math
def find_distance(x, y):
    """Function to get the distance between 2 points on N-dimensions, 
    where x and y are the n-dimensional coordinates of two points,
    it returns the distance"""
    summed = 0                 
    for i in range(len(x)): 
       difference = (x[i] - y[i])**2 
       summed += difference  
    rooted = math.sqrt(summed) 
    return rooted

#testing the function on two, three-dimensional points a and b  
a = [0, 3, 0]
b = [4, 0, 0]

print("The distance between a and b is: ", str(find_distance(a, b)))

### Step 2: Function to estimate the centroid of a collection of points

Great, now we know how to find the distance between two points. This will help up when we want the algorithm to decide to which cluster it will assign the points, it will measure the distance with that function and choose accordingly! 

We now want to write a function called `find_centroid`, which you guessed it, will estimate the centroid for a given selection of n-dimensional points. it will take one list as argument (which contains all of the points entered as a list of n-coordinates). We want it to return a list with the coordinates of their "center point". More on why that makes sense later (the coordinate of the centroid in each dimension is the mean of coordinates of all the points in that dimension). 

We can test our function on `test_lst`: 

In [ ]:
test_lst = [[0,0,0], [0,0,1], [0,1,0], [1,0,0], 
            [0,1,1], [1,0,1], [1,1,0], [1,1,1]]

Now let's write our function: 

In [ ]:
# Equation: sum(points)/n-points
# define the total dimensions in given point
# use list comprehension taking the mean accross points, accross dimensions

def find_centroid(point):
    """Function that returns the centroid, 
    takes argument point, a list of points. 
    Returns a list of mean coordinate per dimension."""

    total_dimensions = len(point[0]) 
    return [sum(coord[i] for coord in point) / len(point) for i in range(total_dimensions)]
    
find_centroid(test_lst)

### Step 3: Function to read data

This doesn't need to be a function, but since we're making functions for everything, it's good practice to have standard, modular functions that you can re-use between projects! I like to keep mine in separate modules so that I can call them when I need to. 

We'll call this function `get_data` and we want it to return the data in a list (can you see the list pattern here?) We'll be using the [csv module](https://docs.python.org/3/library/csv.html) for this. 

Note: we want the list to be for each customer's annual spending on a variety of products, that means we have to get rid of some headings. 

In [ ]:
import csv
# Skipping header, removing columns 1&2 
# Storing data as integers
def get_data():
    """Function returns the data set, skipping its header and the first two columns.
    It takes no argument. 
    The elements in the data are store as integers."""
    
    with open('../data/Wholesale customers data.csv', "r") as wholesale:
        wholesale = csv.reader(wholesale)
        next(wholesale, None) 
        data = [[int(i) for i in line[2:]] for line in wholesale] 
    return data

# assigning a variable to the function to then visualize the output
data = get_data()

#printing the first two elements of the data 
print(data[:2])

### Step 4: Function to implement k-means algorithm

We'll write our final function `kmeans` that clusters a collection of points into k clusters using a simplified version of the k-means algorithm. The function will take two arguments: 

1. `points` – a list of n-dimensional points, and
2. `k` – an integer that defines the number of desired clusters. 

The function will return two things: 

1. A clustering – a list of `k` clusters, each of which is a list of points (each of which is a list of coordinates)
2. A list of the centroids for each of the `k` clusters. Each centroid is essentially a point, so it should be presented as a list of coordinates.

We'll test the function on 3 clusters so that it doesn't take too long to run. 

In [ ]:
import random

def kmeans(points, k): 
    """This function returns clusters and centroids for the input data as lists. 
    It takes the data and the number of desired clusters as arguments.
    It calls the function cluster_points."""
    
    # Select k random points to use as initial centroids
    init = random.sample(points, k)

    # Create a list of k lists to contain the points assigned to each cluster.  
    clusters = [[] for i in init]
     
    # Create a list to keep the centroids of the k clusters. 
    # For now, this list will contain the points from init.

    centroids = [i for i in init]
    
    # You now need to assign each point to the cluster 
    # with the closest centroid. Use the find_distance function 
    # you wrote in Problem 1 for this.
    
    # Function that will create temporary storage of clusters(list of empty lists) according to smallest distance
    # It will iterate each point in the list
    # Infinity = starting point for distance (gets updated as we find the minimum distance)
    # index = 0 starting point to record position of the minimum distance
    # This is more memory efficient than storing all the distances, and finding their minimum 


    def cluster_points(centr):
        """This function returns 'temporary' clusters for the data in a list. 
        It takes for input list of points in kmeans function.
        Clusters are stored as lists."""

        clusters_temp = [[] for i in init]
        for point in points: 
            min_dist = math.inf
            indx = 0
            for i in range(len(centr)):
                dist = find_distance(centr[i], point)    
                if dist < min_dist:
                    min_dist = dist
                    indx = i
            clusters_temp[indx].append(point) 
        return clusters_temp
        
    # You should then update the variable "clusters" to be 
    # the new clustering and update the variable "centroids" 
    # to contain the centroids of the clusters in this new clustering.
    # Use the function you wrote in Problem 2 to estimate the centroids.

    # Repeat the process described above for 100 iterations. 
    # The idea is that each new repetition refines the clustering 
    # because it starts from the centroids of the previous clustering. 
    # If we repeat the process long enough, the assignment to 
    # clusters and the centroids will become stable.

  
    # calling cluster_points function 100 times to make the final clusters 
    # calling the find_centroid function 
    # this will "update" the variable clusters, using the temporary clusters stored above
    # this will get the centroids of the new clusters 

    for i in range(100):
        clusters = cluster_points(centroids) 
        centroids = [find_centroid(cluster) for cluster in clusters] 
    
    return clusters, centroids

clusters, centroids = kmeans(data, 3)

# printing the output per cluster for number customers and centroid of cluster
# I use "i+1" here to number the clusters starting at 1

for i in range(len(clusters)):
    print("There are ", str(len(clusters[i])), " customers in cluster", i+1, ".") 
    print("Its centroid coordinates are: ", centroids[i], ".")


And that's it! We made our simplified k-means clustering algorithm! 